# Google Mount

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

path = '작업할 경로'
os.chdir(path)

# Mecab

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab/

In [ ]:
! bash install_mecab-ko_on_colab190912.sh

In [ ]:
cd /content/mecab-ko-dic-2.1.1-20180720/

In [ ]:
!pip install jamo

In [ ]:
from jamo import h2j, j2hcj

def get_jongsung_TF(sample_text) :
    sample_text_list = list(sample_text)
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word)))
    last_jamo = last_word_jamo_list[-1]
    
    jongsung_TF = "T"

    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ'] :
        jongsung_TF = "F"

    return jongsung_TF

In [ ]:
with open("./user-dic/nng.csv", 'w', encoding='utf-8') as f:
    word_list = ['씨디', '랍스터', '고양이과', '육식동물', '정신의학', '수집품', '독립체', '인종차별', '유사도', '근접성', '중요성', 'FBI', 'OPEC']

    for word in word_list:
        jongsung_TF = get_jongsung_TF(word)
        f.write(line)


with open("./user-dic/va.csv", 'w', encoding='utf-8') as f:
    word_list = ['똑똑한']

    for word in word_list:
        jongsung_TF = get_jongsung_TF(word)
        f.write(line)

In [ ]:
with open("./user-dic/nng.csv", 'r', encoding='utf-8') as f:
    file_new = f.readlines()

print(file_new)


with open("./user-dic/va.csv", 'r', encoding='utf-8') as f:
    file_new = f.readlines()

print(file_new)

In [ ]:
!bash ./tools/add-userdic.sh

In [ ]:
!make install

In [ ]:
with open("./user-nng.csv", 'r', encoding='utf-8') as f:
    file_data1 = f.readlines()
    print(file_data1)

with open("./user-va.csv", 'r', encoding='utf-8') as f:
    file_data2 = f.readlines()
    print(file_data2)

In [ ]:
len(file_data1)
file_data1[10] = '중요성,1786,3546,0,NNP,*,T,중요성,*,*,*,*,*\n'
file_data1[11] = 'FBI,1786,3546,0,NNP,*,T,FBI,*,*,*,*,*\n'
file_data1[12] = 'OPEC,1786,3546,0,NNP,*,T,OPEC,*,*,*,*,*\n'
print(file_data1)

file_data2[0] = '똑똑한,1786,3546,0,VA,*,T,똑똑한,*,*,*,*,*\n'
print(file_data2)

In [ ]:
with open("./user-nng.csv", 'w', encoding='utf-8') as f:
    for line in file_data1 :
        f.write(line)

with open("./user-va.csv", 'w', encoding='utf-8') as f:
    for line in file_data2 :
        f.write(line)

In [ ]:
with open("./user-nng.csv", 'r', encoding='utf-8') as f:
    file_data1 = f.readlines()
    print(file_data1)

with open("./user-va.csv", 'r', encoding='utf-8') as f:
    file_data2 = f.readlines()
    print(file_data2)

In [ ]:
!make clean

In [ ]:
!make install

In [ ]:
# 제대로 추가되어 형태소 분석시 잘 분석되는지 확인
from konlpy.tag import Mecab

mecab = Mecab()

word_list = word_list = ['씨디', '랍스터', '고양이과', '육식동물', '정신의학', '수집품', '독립체', '인종차별', '유사도', '근접성', '중요성', "똑똑한", 'FBI', OPEC]

for word in word_list:
    print(mecab.pos(word))

In [ ]:
# 다시 작업할 폴더로 이동
cd 작업할경로

# 전처리

In [ ]:
import pandas as pd
import re

chunk = pd.read_table("./data/wiki_ko_mecab.txt", sep='\n', encoding='utf-8', header=None, chunksize = 1000)

chunk = list(chunk)

for df in chunk :
    df = pd.concat(chunk)
	
df.columns = ["doc"]

In [ ]:
def fix_keyerror_word(string: str, key_error_set: list, fix_set: list) :
    if any(word in string for word in key_error_set) :
        i = len(key_error_set) - 1

        def fix_keyerror(i) :
            nonlocal string
            if i == -1 :
                return string
            string = re.sub(key_error_set[i], fix_set[i],string)
            i -= 1
            fix_keyerror(i)

        fix_keyerror(i)
        
        return string
    else :
        return string

key_error_set = [" 씨 디 ", " 똑똑 한 ", " 랍스 터 ", " 고양이 과 ", " 육식 동물 ", " 정신 의학 ", " 수집 품 ", " 독립 체 ", " 인종 차별 ", " 유사 도 ", " 근접 성 ", " 중요 성 "]
fix_set = [" 씨디 ", " 똑똑한 ", " 랍스터 ", " 고양이과 ", " 육식동물 ", " 정신의학 ", " 수집품 ", " 독립체 ", " 인종차별 ", " 유사도 ", " 근접성 ", " 중요성 "]


df["fixed_doc"] = df["doc"].apply(fix_keyerror_word, key_error_set=key_error_set, fix_set=fix_set)

In [ ]:
def do_mecab_for_regex(string: str) -> str:
    essential_pos_word_list = []
    only_noun_pos_word_list = []
    gen = (x for x in mecab.pos(string))
    for word_pos in gen :
        if word_pos[1] in ["NNP","NNG", "XPN", "XSN","VV","VA","VX","VCP","VCN","EP","EC","ETN","ETM", "XR", "MM", "MAG","MAJ","XR"] :
            essential_pos_word_list.append(word_pos[0])

            if word_pos[1] in ["NNP","NNG", "XPN", "XSN"] :
                only_noun_pos_word_list.append(word_pos[0])

    return ",".join(essential_pos_word_list), ",".join(only_noun_pos_word_list)

df["essential_pos_doc"], df["only_noun_pos_doc"] = df["fixed_doc"].apply(do_mecab_for_regex)

In [ ]:
df["only_noun_pos_doc"].to_csv("저장할 파일 경로", index=False, header = None, sep = "\n")
df["essential_pos_doc"].to_csv("저장할 파일 경로", index=False, header = None, sep = "\n")